In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, ConcatDataset
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [ ]:
# TODO: Set hyperparameters.
training_batch_size = 20
validation_batch_size = 20
test_batch_size = 102
epochs = 30
learning_rate = 0.001
momentum = 0.9
crop_size = 227

In [ ]:
# Default to CPU
device = torch.device("cpu")

# Switch to GPU if available
if torch.cuda.is_available():
	print(f"Found {torch.cuda.device_count()} GPUs. Using cuda:0.")
	device = torch.device("cuda:0")
else:
	print("No GPUs found, using CPU.")

Found 1 GPUs. Using cuda:0.


In [ ]:
training_data = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

training_data2 = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.RandomHorizontalFlip(p=1),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

training_data3 = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.RandomRotation(degrees=45),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)


training_data4 = datasets.Flowers102(
    root = "data",
    split = "train",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.RandomRotation(degrees=23),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)


validation_data = datasets.Flowers102(
    root = "data",
    split = "val",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

testing_data = datasets.Flowers102(
    root = "data",
    split = "test",
    transform=transforms.Compose([
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ]),
    download=True
)

In [ ]:
horizontal = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1)
])

degrees = transforms.Compose([
    transforms.RandomRotation(degrees=45)
])

In [ ]:
augmented_data = ConcatDataset([training_data, training_data2, training_data3, training_data4])

In [ ]:
train_dataloader = DataLoader(augmented_data, batch_size=training_batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=validation_batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=test_batch_size, shuffle=True)

In [ ]:
class F102Classifier(nn.Module):
    
	def __init__(self):
		super(F102Classifier, self).__init__()
		self.features = nn.Sequential(
			nn.Conv2d(3, 27, 3, 1, 1),
			nn.BatchNorm2d(27),
      nn.ReLU(),
			nn.MaxPool2d(2, 2),
   		nn.Conv2d(27, 81, 3, 1, 1),
      nn.BatchNorm2d(81),
			nn.ReLU(),
   		nn.Conv2d(81, 243, 3, 1, 1),
			nn.BatchNorm2d(243),
      nn.ReLU(),
			nn.MaxPool2d(2, 2),
		)
  
		self.classifier = nn.Sequential(
			nn.Dropout(0.5),
			nn.Linear(243*56*56, 512),
			nn.BatchNorm1d(512),
			nn.ReLU(),
   		nn.Dropout(0.5),
			nn.Linear(512,102)
		)

	def forward(self, x):
		x = self.features(x)
		x = torch.flatten(x)
		x = x.view(training_batch_size, -1)
		x = self.classifier(x)
		return x

net = F102Classifier()

if torch.cuda.is_available():
    net.cuda()

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
	running_loss = 0.0
	for batch, (i,j) in enumerate(dataloader):
		features, labels = i.to(device), j.to(device)
  
		# Compute the loss based off the predictions vs labels
		predictions = model(features)
		loss = loss_fn(predictions, labels)
		running_loss += loss.item()
  
		#Compute back propagation
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		divisor = batch

	running_loss /= divisor
	
	print(f'Average Loss in Epoch: {running_loss}')
			
print('Finished Training')

Finished Training


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for batch, (i,j) in enumerate(dataloader):
        features, labels = i.to(device), j.to(device)
        pred = model(features)
        test_loss += loss_fn(pred, labels).item()
        correct += (pred.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f'Epoch {t+1}-------------')
    train(train_dataloader, net, loss_function, optimiser)
    test(validation_dataloader, net, loss_function)

Epoch 1-------------
Average Loss in Epoch: 3.4832298315217343
Test Error: 
 Accuracy: 24.6%, Avg loss : 3.015719 

Epoch 2-------------
Average Loss in Epoch: 1.6499384136622763
Test Error: 
 Accuracy: 29.8%, Avg loss : 3.277803 

Epoch 3-------------
Average Loss in Epoch: 0.5382211414977834
